# Discrete Gauss-Bonnet-Chern Theorem
## Computational Demonstration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from core.curvature import DiscreteManifold
from core.characteristic_classes import CharacteristicClasses
from topology.poincare_proof import discrete_poincare_proof
from topology.exotic_spheres import ExoticSphereDatabase

# Create different geometric shapes
def create_torus(R=1, r=0.3, n=100):
    u = np.linspace(0, 2*np.pi, n)
    v = np.linspace(0, 2*np.pi, n)
    u, v = np.meshgrid(u, v)
    x = (R + r*np.cos(v)) * np.cos(u)
    y = (R + r*np.cos(v)) * np.sin(u)
    z = r * np.sin(v)
    return np.vstack([x.flatten(), y.flatten(), z.flatten()]).T

def create_sphere(n=100):
    vertices = np.random.randn(n, 3)
    vertices /= np.linalg.norm(vertices, axis=1)[:, np.newaxis]
    return vertices

def create_double_torus(n=100):
    # Placeholder - actual implementation would be more complex
    return create_torus(R=1.5, r=0.4, n=n)

# Create triangulation
def triangulate(vertices):
    from scipy.spatial import Delaunay
    tri = Delaunay(vertices[:, :2])  # Project to 2D for triangulation
    return tri.simplices

# Analyze a shape
def analyze_shape(name, vertices):
    simplices = triangulate(vertices)
    manifold = DiscreteManifold(vertices, simplices, dim=2)
    
    # Compute GBC integral
    gbc_integral = manifold.gbc_integral()
    
    # Compute Euler characteristic
    chi = manifold.euler_characteristic()
    
    # Compute expected GBC
    expected = (2*np.pi) * chi
    
    # Print results
    print(f"\n--- {name} ---")
    print(f"Computed GBC integral: {gbc_integral:.4f}")
    print(f"Expected GBC (2πχ): {expected:.4f}")
    print(f"Euler characteristic χ: {chi}")
    print(f"Error: {abs(gbc_integral - expected)/expected:.2%}")
    
    return manifold

# Analyze different surfaces
sphere = analyze_shape("Sphere", create_sphere(200))
torus = analyze_shape("Torus", create_torus(n=50))
double_torus = analyze_shape("Double Torus", create_double_torus(n=150))

# 4D Poincaré Proof Demonstration
print("\n=== 4D Poincaré Proof ===")
db = ExoticSphereDatabase()
standard_sphere = db.get_sphere("S4_standard")
exotic_sphere = db.get_sphere("exotic_0")

if standard_sphere and exotic_sphere:
    print("Standard Sphere:")
    print(f"- Eta invariant: {standard_sphere.obstruction_value():.6f}")
    print(f"- Obstruction: {db.obstruction_check(standard_sphere)}")
    
    print("\nExotic Sphere:")
    print(f"- Eta invariant: {exotic_sphere.obstruction_value():.6f}")
    print(f"- Obstruction: {db.obstruction_check(exotic_sphere)}")
    
    # Check Poincaré conditions
    print("\nDiscrete Poincaré Proof:")
    print(f"Standard sphere: {discrete_poincare_proof(standard_sphere)}")
    print(f"Exotic sphere: {discrete_poincare_proof(exotic_sphere)}")
else:
    print("Exotic sphere database not populated. Run exotic_spheres.py first.")

# Characteristic Classes
print("\n=== Characteristic Classes ===")
cc = CharacteristicClasses(sphere)
print(f"Euler class: {cc.compute_euler_class():.4f}")
if sphere.dim == 4:
    print(f"Signature: {cc.compute_signature():.4f}")

# Visualization
def plot_curvature(manifold, title):
    defects = manifold.compute_angle_defect()
    vertex_defects = np.zeros(len(manifold.vertices))
    for hinge, defect in defects.items():
        for v in hinge:
            vertex_defects[v] += defect
    
    plt.figure(figsize=(10, 8))
    plt.tricontourf(
        manifold.vertices[:,0], 
        manifold.vertices[:,1], 
        manifold.simplices, 
        vertex_defects, 
        cmap='viridis'
    )
    plt.colorbar(label='Curvature')
    plt.title(f"Curvature Distribution: {title}")
    plt.show()

plot_curvature(sphere, "Sphere")
plot_curvature(torus, "Torus")